### Simpler webscrapping no need for no-driver

But if authentication was oauth2 I would probably mix no-driver with bellow

In [25]:
from curl_cffi import requests # very fast and equivalent to requests
from selectolax.parser import HTMLParser

In [26]:
anm_user, anm_passwd = 'x.x', 'x@x'

In [72]:
from urllib.parse import urlparse, parse_qs

def print_query_string_params(url):
    parsed_url = urlparse(url)
    print(f'url {url}')
    print(f'parsed_url, netloc {parsed_url.netloc} path {parsed_url.path}')
    query_params = parse_qs(parsed_url.query)
    for key, value in query_params.items():
        print(f"{key}: {value}")

def debug_response(response, ignore_headers=True):
    print("Response Headers:")
    print_query_string_params(response.url)
    if not ignore_headers:
        for header, value in response.headers.items():
            print(f"{header}: {value}")
    with open('response.html', 'w') as f:
        f.write(response.text)
    return response


#### Log-in

In [68]:
url = 'https://sei.anm.gov.br/sip/login.php?sigla_orgao_sistema=ANM&sigla_sistema=SEI'

session = requests.Session(impersonate='chrome120')
session.verify='/home/andre/Documents/Controle_Areas/secorpy/sei-gov-br-chain.pem'    
response = session.get(url)
html = HTMLParser(response.content)
hdntoken = html.css_first('input[type="hidden"][id^="hdnToken"]')
hdntoken.attributes['value']
response.raise_for_status()
formdata = {
    'txtUsuario': anm_user,
    'pwdSenha': anm_passwd, 
    'selOrgao': '26',
    'chkLembrar': 'on',
    'sbmLogin': 'Acessar',
    hdntoken.attributes['name']: hdntoken.attributes['value']
}
response = session.post(url, data=formdata)
response.raise_for_status()   
debug_response(response)

Response Headers:
parsed_url, netloc sei.anm.gov.br path /sei/controlador.php
acao: ['procedimento_controlar']
acao_origem: ['principal']
acao_retorno: ['principal']
inicializando: ['1']
infra_sistema: ['100000100']
infra_unidade_atual: ['110000749']
infra_hash: ['04058e515bac17baa64fa627872d0116ce3b802d652b5a8286a8a5cab16f857c']


<Response [200]>

### Get Processo

In [78]:
html = HTMLParser(response.content)
pesquisa = html.css_first('#frmProtocoloPesquisaRapida')
pesquisa.attributes['action']
formdata = { 'txtPesquisaRapida' : '48054.832390/2024-66' } 
pesquisa_url = 'https://sei.anm.gov.br/sei/' + pesquisa.attributes['action']
response = session.post(pesquisa_url, data=formdata)

In [79]:
debug_response(response)

Response Headers:
url https://sei.anm.gov.br/sei/controlador.php?acao=procedimento_trabalhar&acao_origem=protocolo_pesquisa_rapida&id_protocolo=16592114&infra_sistema=100000100&infra_unidade_atual=110000749&infra_hash=970e091e6c142ac916729841ba720624d17ea243a971e8c00b81b4de3485def1
parsed_url, netloc sei.anm.gov.br path /sei/controlador.php
acao: ['procedimento_trabalhar']
acao_origem: ['protocolo_pesquisa_rapida']
id_protocolo: ['16592114']
infra_sistema: ['100000100']
infra_unidade_atual: ['110000749']
infra_hash: ['970e091e6c142ac916729841ba720624d17ea243a971e8c00b81b4de3485def1']


<Response [200]>

### Visualizar Processo

In [77]:
#response = session.get(response.url)

In [80]:
html = HTMLParser(response.content)
visualizar = html.css_first('iframe#ifrArvore')
visualizar_url = 'https://sei.anm.gov.br/sei/' + visualizar.attributes['src']
response = session.get(visualizar_url)

In [81]:
debug_response(response)

Response Headers:
url https://sei.anm.gov.br/sei/controlador.php?acao=procedimento_visualizar&acao_origem=procedimento_trabalhar&acao_retorno=principal&id_procedimento=16592114&id_documento=&id_procedimento_anexado=&infra_sistema=100000100&infra_unidade_atual=110000749&infra_hash=97a74bf04c5f9ff30192766817ab590b8cf57e31649ea300622e262a3ff39fbe
parsed_url, netloc sei.anm.gov.br path /sei/controlador.php
acao: ['procedimento_visualizar']
acao_origem: ['procedimento_trabalhar']
acao_retorno: ['principal']
id_procedimento: ['16592114']
infra_sistema: ['100000100']
infra_unidade_atual: ['110000749']
infra_hash: ['97a74bf04c5f9ff30192766817ab590b8cf57e31649ea300622e262a3ff39fbe']


<Response [200]>

In [12]:
print(response)

Headers({'Date': 'Tue, 18 Mar 2025 23:55:09 GMT', 'Server': 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips PHP/5.6.33', 'X-Powered-By': 'PHP/5.6.33', 'Expires': '0', 'Cache-Control': 'private, no-cache, no-store, must-revalidate, max-age=0', 'Pragma': 'no-cache', 'X-Robots-Tag': 'noindex', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=iso-8859-1'})
